In [ ]:
# Librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

In [ ]:
df = pd.read_csv('/datasets/df_songs.csv')

*Mode: True = Major Scale / False = Minor Scale*

In [ ]:
df.head()

In [ ]:
df.describe()

**Duda:** ¿podría haber un problema con el escalado de year al introducir nuevos datos?

# Training

In [ ]:
# Se decidió descartar 'mode ya que, al ser incluida en el dataset, las predicciones siempre coinciden con el "mode" original (Mayor o menor). Pero los oyentes no escuchan exclusivamente música de un modo, sino que consumen ambos modos de forma indistinta.
df.drop(columns=['mode'],inplace=True)

In [ ]:
# Selecting relevant audio features
features = ['track_popularity', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'year']
# ¿incluir track popularity?
# ¿¿ Eliminación 'acousticness' por alta correlación con 'energy' y 'loudness' ??
# Reintroducir feature: mode

X = df[features].values

In [ ]:
X = X.astype('float32')

In [ ]:
# Splitting the data
X_train, X_test = train_test_split(X, test_size=0.3, random_state=42)

In [ ]:
model = Sequential([
    Input(shape=(len(features),)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu', name="embedding_layer"), # Embedding layer
    Dense(16, activation='relu'),
    Dense(len(features), activation='linear') # Output layer for reconstruction
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [ ]:
# Entrenamiento del modelo y guardar el historial
history = model.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test))

In [ ]:
# Graficar la pérdida de entrenamiento y validación
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Curva de Entrenamiento y Validación')
plt.show()

In [ ]:
# Define model with the functional API
input_layer = Input(shape=(len(features),))
x = Dense(128, activation='relu')(input_layer)
x = Dropout(0.3)(x)
embedding_layer = Dense(64, activation='relu', name="embedding_layer")(x)  # Named embedding layer
x = Dropout(0.3)(embedding_layer)
x = Dense(32, activation='relu')(x)
output_layer = Dense(len(features), activation='linear')(x)

# Compile the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Initialize the model by running a sample input through it
sample_input = np.random.rand(1, len(features))  # Sample input
_ = model.predict(sample_input)  # Initialize the model

# Extract embeddings using the embedding layer directly
embedding_model = Model(inputs=model.input, outputs=model.get_layer("embedding_layer").output)
song_embeddings = embedding_model.predict(X)

print("Embeddings shape:", song_embeddings.shape)


In [ ]:
# Exportación del modelo
embedding_model.save("/kaggle/working/songs_embeddings.h5")

### Búsqueda por género

In [ ]:
# Índices para facilitar la búsqueda por género
genres = ['pop', # 0
          'rock', # 1
          'rap', # 2
          'r&b', # 3
          'edm', # 4
          'latin'] # 5

df[df['playlist_genre'] == genres[1]]  # Selecciona sólo un género
#df[df['playlist_genre'] != genres[1]]  # Todos los demás géneros

### Búsqueda por artista/canción

In [ ]:
# Búsqueda por artista/nombre
df.loc[df['track_artist'] == 'Daft Punk']
#df.loc[df['track_name'] == 'Yesterday']

### Búsqueda por ID

Ejemplos de testing:
- 3gu0fRSgFuc4FmrtIv0DnC (Reggaeton)
- 0r7CVbZTWZgbTCYdfa2P31 (pop)
- 5yY9lUy8nbvjM1Uyo1Uqoc (rap)

In [ ]:
# Búsqueda por ID
id_ = "5CMjjywI0eZMixPeqNd75R"
name = df.loc[df['track_id'] == id_]
song_index = df[df['track_id'] == id_].index.item()
print(name.iloc[0]['track_name'], 'by', name.iloc[0]['track_artist'], 'on album', name.iloc[0]['track_album_name'])
#print(song_index)
print()
df.iloc[song_index]

In [ ]:
# Búsqueda por index
#song_index = 12616  # For example, the first song in the dataset
"""
name = df.loc[df['track_id'] == df.iloc[song_index]['track_id']]
print(name.iloc[0]['track_name'], 'by', name.iloc[0]['track_artist'], 'on album', name.iloc[0]['track_album_name'])
print()

df.iloc[song_index]
"""

## Predicción

In [ ]:
num_pred = 5 # Modificar a gusto para obtener más predicciones
similarities = cosine_similarity([song_embeddings[song_index]], song_embeddings)[0]

# Get top n most similar songs (excluding itself)
similar_indices = np.argsort(similarities)[-(num_pred+1):-1][::-1]
similar_songs = df.iloc[similar_indices]

for i, song in enumerate(similar_songs.index):
    print(f"""{i+1}º) "{similar_songs.loc[song, 'track_name']}" by {similar_songs.loc[song, 'track_artist']} on album "{similar_songs.loc[song, 'track_album_name']}" ({similar_songs.loc[song, 'playlist_genre'].title()})\n""")

In [ ]:
similar_songs

## Análisis de resultados

In [ ]:
# 0. Canción original (user input)
song = df.iloc[song_index].copy()
song = song.drop(index=['track_id','track_name','track_artist','playlist_genre',
                       'track_album_id','track_album_name'])
song = song.astype(float)

# 0. Calcular la media de las características en el dataframe completo
df_mean = df[features].mean()

# 1. Filtrar las canciones similares usando los índices y calcular la media
similar_mean = similar_songs[features].mean()

# 2. Crear un DataFrame que contenga ambas medias para fácil comparación
comparison_df = pd.DataFrame({'Original song': song, 'Similar songs': similar_mean})

# 3. Calcular el RMSE entre la canción original y la media de las canciones similares
comparison_df['Similarity (RMSE)'] = np.sqrt((comparison_df['Original song'] - comparison_df['Similar songs']) ** 2)
comparison_df['Other songs'] = df_mean

# 4. Calcular el RMSE entre la media de canciones similares y la media general
comparison_df['Difference (RMSE)'] = np.sqrt((comparison_df['Similar songs'] - comparison_df['Other songs']) ** 2)

# 5. Compara el RMSE de la predicción contra la media del dataset (Normalizado)
comparison_df['Eficacia (%)'] =  comparison_df['Difference (RMSE)'] / (comparison_df['Similarity (RMSE)'] + comparison_df['Difference (RMSE)'])
# Valores cercanos a 1 indican una predicción excelente
# Valores cercanos a 0 indican una predicción pobre (RMSE similar entre la predicción y la media)

In [ ]:
comparison_df

**NOTAS:**
- Similarity/Diference: representan la proximidad de los datos a través del RMSE (0 es "similar" | 1 es "diferente")
- Eficacia: 1 indica una predicción excelente. 0 indica una predicción pobre.

In [ ]:
# Creating an explicit axis for the colorbar to resolve the deprecation warning
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the color map from blue (near 0) to red (near 1) for Eficacia
norm = plt.Normalize(0, 1)
sm = plt.cm.ScalarMappable(cmap="coolwarm", norm=norm)
sm.set_array([])  # Setting an empty array to avoid warnings

# Create the figure and plot each bar with color based on Eficacia value
fig, ax = plt.subplots(figsize=(15, 6))
bars = ax.bar(comparison_df.index, comparison_df['Eficacia (%)'], color=sm.to_rgba(comparison_df['Eficacia (%)']))

# Customize the x-axis labels to be angled at 45 degrees and smaller font size
ax.set_xticks(comparison_df.index)
ax.set_xticklabels(features, rotation=45, ha='right', fontsize=10)

# Labels and title
ax.set_xlabel("Features")
ax.set_ylabel("Eficacia (Normalized between 0 and 1)")
ax.set_title("Normalized Eficacia for Each Feature")

# Add color bar with a specified axis
cbar = fig.colorbar(sm, ax=ax, orientation='vertical', pad=0.02)
cbar.set_label('Eficacia scale (0: Blue, 1: Red)')

plt.show()


In [ ]:
threshold = 0.6
indices = comparison_df[comparison_df['Eficacia (%)'] > threshold].index
top_features = comparison_df.loc[indices].sort_values(by="Eficacia (%)", ascending=False)
print("Eficacia de los features representativos:")
for i, (feature, row) in enumerate(top_features['Eficacia (%)'].items(), start=1):
    print(f"{i}º) {feature}: {row*100:.1f}%")
    
#print(top_features['Eficacia'])

**Ideas:**
- Se puede ponderar el peso de ciertas features en la búsqueda (por ejemplo el año)
- Se puede incluir un feature que indique el idioma de la letra.

**Observaciones:**
- Es probable que mode no sea de utilidad. Al ser incluida en el dataset, la predicción siempre coincide con el "mode" original, pero los oyentes no escuchan exclusivamente música de un modo.
- Pareciera que la búsqueda de similitud tiende a evitar outliers.